# MathorCup 2024 论文支撑材料
## YinMo19

In [74]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

from scipy.integrate import odeint
from scipy import linalg as la
from scipy import optimize
import scipy
import csv
import pandas as pd

config = {
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage{CJK}",  # 预先导入CJK宏包处理中文
}
plt.rcParams.update(config)

## 读入数据

In [75]:
data = []
with open("../../附件/附件1.csv", "r", encoding="gb2312") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data.append(row)
        
head = data[0]

In [76]:
ALL_SC = list(set(list(np.array(data)[1:,0])))
data = np.array(data)

各种数据的声明：
- `data`：所有数据，以二重列表的形式，将每一行作为一个列表。
- `head`：表头
- `ALL_SC`：无序的所有`SC__`的合集
- `datas`: 货物量，以`SC__`为分类方法

将数据分开

In [77]:
datas = np.array(
    [[data__[2] for data__ in data[1:] if data__[0] == SC_] for SC_ in ALL_SC],
    dtype="int64",
)

写入csv文件，分开储存

In [78]:
for SC_ in ALL_SC:
    with open(f"../SC_datas/{SC_}.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(head)
        
        for row in data:
            if row[0] == SC_:
                writer.writerow(row)

## 箱线图绘制

In [79]:
# 创建图表
fig, ax = plt.subplots(2, 3,figsize=(18, 10))
ax[0][0].boxplot(list(datas[0:9]), patch_artist=True, sym="+", labels=ALL_SC[0:9])
ax[0][1].boxplot(list(datas[10:19]), patch_artist=True, sym="+", labels=ALL_SC[10:19])
ax[1][0].boxplot(list(datas[20:29]), patch_artist=True, sym="+", labels=ALL_SC[20:29])
ax[1][1].boxplot(list(datas[30:39]), patch_artist=True, sym="+", labels=ALL_SC[30:39])
ax[0][2].boxplot(list(datas[40:49]), patch_artist=True, sym="+", labels=ALL_SC[40:49])
ax[1][2].boxplot(list(datas[50:56]), patch_artist=True, sym="+", labels=ALL_SC[50:56])

# 显示图表
with PdfPages("images/xxt_1.pdf") as pdf:
    pdf.savefig()
    plt.close()

清洗数据之后的箱线图

In [80]:
# 创建图表
fig, ax = plt.subplots(2, 3,figsize=(18, 10))
ax[0][0].boxplot(list(datas[0:9]), patch_artist=True, sym="", labels=ALL_SC[0:9])
ax[0][1].boxplot(list(datas[10:19]), patch_artist=True, sym="", labels=ALL_SC[10:19])
ax[1][0].boxplot(list(datas[20:29]), patch_artist=True, sym="", labels=ALL_SC[20:29])
ax[1][1].boxplot(list(datas[30:39]), patch_artist=True, sym="", labels=ALL_SC[30:39])
ax[0][2].boxplot(list(datas[40:49]), patch_artist=True, sym="", labels=ALL_SC[40:49])
ax[1][2].boxplot(list(datas[50:56]), patch_artist=True, sym="", labels=ALL_SC[50:56])

# 显示图表
with PdfPages("images/xxt_2.pdf") as pdf:
    pdf.savefig()
    plt.close()

清洗数据之后的csv

In [97]:
for SC_ in ALL_SC:
    # 读取CSV数据
    data = pd.read_csv(
        f"../SC_datas/{SC_}.csv", header=None, names=["分拣中心", "日期", "货量"]
    )

    data["货量"] = pd.to_numeric(data["货量"], errors="coerce")
    data = data.dropna(subset=["货量"])

    # 根据异常值标准删除行
    q1 = data["货量"].quantile(0.25)
    q3 = data["货量"].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    data = data[(data["货量"] >= lower_bound) & (data["货量"] <= upper_bound)]

    # 将处理后的数据写入新的CSV文件
    data.to_csv(f"../processed_SC/processed_{SC_}.csv", index=False)